In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt


# List of tickers to analyze
tickers = ["DPST", "RYCEY", "KO", "ADA-USD"]  # Replace with your desired stock tickers

# Initialize an empty dictionary to store results
results = {}

# Function to calculate technical indicators
def calculate_indicators(data):
    print("Calculating indicators...")

    # Convert index datetime to date
    data.index = data.index.date

    # Bollinger Bands
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['Upper_Band'] = data['20_SMA'] + (data['Close'].rolling(window=20).std() * 2)
    data['Lower_Band'] = data['20_SMA'] - (data['Close'].rolling(window=20).std() * 2)

    # On-Balance Volume (OBV)
    data['Daily_Change'] = data['Close'].diff()
    data['Direction'] = np.where(data['Daily_Change'] > 0, 1, np.where(data['Daily_Change'] < 0, -1, 0))
    data['OBV'] = (data['Volume'] * data['Direction']).cumsum()

    # Stochastic Oscillator
    low_14 = data['Low'].rolling(14).min()
    high_14 = data['High'].rolling(14).max()
    data['%K'] = ((data['Close'] - low_14) / (high_14 - low_14)) * 100
    data['%D'] = data['%K'].rolling(window=3).mean()

    # MACD
    data['MACD'] = data['Close'].ewm(span=12).mean() - data['Close'].ewm(span=26).mean()
    data['Signal'] = data['MACD'].ewm(span=9).mean()

    # RSI
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # ATR
    data['ATR'] = (data['High'] - data['Low']).rolling(14).mean()

    # Target: Predict if the price will increase in the next 5 days
    data['Target'] = np.where(data['Close'].shift(-5) > data['Close'], 1, 0)

    print("Indicators calculated!")
    return data.dropna()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set max width for wrapping
pd.set_option('display.float_format', '{:.2f}'.format)  # Format floats to 2 decimal places


# Loop through each ticker
for ticker in tickers:
    print(f"\nProcessing {ticker}...")

    # Fetch historical data
    stock_data = yf.Ticker(ticker).history(period="1y")
    if stock_data.empty:
        print(f"No data found for {ticker}. Skipping.")
        continue

    # Calculate indicators
    stock_data = calculate_indicators(stock_data)

    # Debugging: Check calculated features
    print(f"Sample feature values for {ticker}:")
    # Print the data
    print(stock_data[['20_SMA', 'Upper_Band', 'Lower_Band', 'OBV', '%K', '%D', 'MACD', 'Signal', 'RSI', 'ATR']].head(3).T)
    plt.figure(figsize=(14, 6))  # Width: 14 inches, Height: 6 inches
    sns.heatmap(stock_data[['20_SMA', 'Upper_Band', 'Lower_Band', 'OBV', '%K', '%D', 'MACD', 'Signal', 'RSI', 'ATR']].head(3), annot=True, fmt=".2f", cmap="YlGnBu")
    plt.title(f"{ticker} Indicators Heatmap")
    plt.show()

    # Skip if there is insufficient data
    if stock_data.shape[0] < 50:
        print(f"Not enough data for {ticker}. Skipping.")
        continue

    # Define features and target
    features = ['MACD', 'Signal', 'RSI', 'ATR', 'Upper_Band', 'Lower_Band', 'OBV', '%K', '%D']
    X = stock_data[features]
    y = stock_data['Target']

    # Debugging: Check target distribution
    print(f"Target distribution for {ticker}:")
    print(y.value_counts())

    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train Random Forest Classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{ticker} Model Accuracy: {accuracy*100:.2f}")

    # Feature Importance
    importances = model.feature_importances_
    print(f"Feature importance for {ticker}:")
    for feature, importance in zip(features, importances):
        print(f"  {feature}: {importance*100:.2f}%")

    # Predict the next movement
    latest_data = stock_data.iloc[-1:][features]
    latest_data_scaled = scaler.transform(latest_data)
    prediction = model.predict(latest_data_scaled)[0]
    probability = model.predict_proba(latest_data_scaled)[0][1]

    # Store the result
    recommendation = "Buy" if prediction == 1 else "Sell"
    reentry_price = stock_data['Close'].iloc[-1] * 0.9 if prediction == 0 else None
    results[ticker] = {
        "Recommendation": recommendation,
        "Probability": probability,
        "Current Price": stock_data['Close'].iloc[-1],
        "Reentry Price": reentry_price
    }

# Display results
for ticker, result in results.items():
    print(f"\n{ticker} Recommendation:")
    print(f"  - Recommendation: {result['Recommendation']}")
    print(f"  - Probability of Upward Movement: {result['Probability']*100:.2f}%")
    print(f"  - Current Price: ${result['Current Price']:.2f}")
    if result['Reentry Price']:
        print(f"  - Suggested Reentry Price: ${result['Reentry Price']:.2f}")


In [2]:
import yfinance as yf
stock = yf.Ticker("DOGE-USD")
hist = stock.history(period="max")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-11-09 00:00:00+00:00,0.001207,0.001415,0.001181,0.001415,6259550,0.0,0.0
2017-11-10 00:00:00+00:00,0.001421,0.001431,0.001125,0.001163,4246520,0.0,0.0
2017-11-11 00:00:00+00:00,0.001146,0.001257,0.001141,0.001201,2231080,0.0,0.0
2017-11-12 00:00:00+00:00,0.001189,0.001210,0.001002,0.001038,3288960,0.0,0.0
2017-11-13 00:00:00+00:00,0.001046,0.001212,0.001019,0.001211,2481270,0.0,0.0
...,...,...,...,...,...,...,...
2025-08-03 00:00:00+00:00,0.191214,0.199928,0.189542,0.198864,1383820507,0.0,0.0
2025-08-04 00:00:00+00:00,0.198863,0.211188,0.198653,0.210413,1808700824,0.0,0.0
2025-08-05 00:00:00+00:00,0.210413,0.210878,0.196447,0.199739,1606059547,0.0,0.0


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

# Step 1: Select candidate stocks under $10
tickers = ["AAL", "GE", "F", "BAC", "PLUG", "NIO", "CCL", "AMC", "SNDL",
            "SOFI", "GM", "ET", "APA", "MO", "BWA", "SYF", "DAL", "AES"]  # Add low-cost stocks
data = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y")
    if not hist.empty:
        hist["Ticker"] = ticker
        data[ticker] = hist

# Combine all data into one DataFrame
all_data = pd.concat(data.values(), keys=data.keys(), axis=0)
all_data.reset_index(inplace=True)

# Step 2: Calculate technical indicators for all stocks
def add_indicators(df):

    # Bollinger Bands
    df['20_SMA'] = df['Close'].rolling(window=20).mean()
    df['Upper_Band'] = df['20_SMA'] + (df['Close'].rolling(window=20).std() * 2)
    df['Lower_Band'] = df['20_SMA'] - (df['Close'].rolling(window=20).std() * 2)

    # On-Balance Volume (OBV)
    df['Daily_Change'] = df['Close'].diff()
    df['Direction'] = np.where(df['Daily_Change'] > 0, 1, np.where(df['Daily_Change'] < 0, -1, 0))
    df['OBV'] = (df['Volume'] * df['Direction']).cumsum()

    # Stochastic Oscillator
    low_14 = df['Low'].rolling(14).min()
    high_14 = df['High'].rolling(14).max()
    df['%K'] = ((df['Close'] - low_14) / (high_14 - low_14)) * 100
    df['%D'] = df['%K'].rolling(window=3).mean()

    # MACD
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal'] = df['MACD'].ewm(span=9).mean()

    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # ATR
    df['ATR'] = (df['High'] - df['Low']).rolling(14).mean()

    # Target: Predict if the price will increase in the next 5 days
    df['Target'] = np.where(df['Close'].shift(-5) > df['Close'], 1, 0)
    return df

all_data = all_data.groupby('Ticker').apply(add_indicators)
all_data.dropna(inplace=True)

# Step 3: Train model
features = ['MACD', 'Signal', 'RSI', 'ATR', 'Upper_Band', 'Lower_Band', 'OBV', '%K', '%D']
X = all_data[features]
y = all_data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Step 4: Make predictions for each stock
predictions = {}
for ticker in tickers:
    df = all_data[all_data['Ticker'] == ticker]
    if not df.empty:
        last_row = df.iloc[-1:][features]  # Ensure last_row remains a DataFrame
        last_row_scaled = scaler.transform(last_row)
        prob = model.predict_proba(last_row_scaled)[0][1]  # Probability of upward movement
        predictions[ticker] = prob

# Step 5: Select top 3 stocks based on highest probability
selected_stocks = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:3]

# Step 6: Fetch current prices and ensure total cost is under $100
selected_stock_details = []
total_cost = 0
for stock, prob in selected_stocks:
    price = yf.Ticker(stock).history(period="1d")['Close'].iloc[-1]
    shares_to_buy = max(int((100 - total_cost) // price), 1)  # Ensure we can afford it
    cost = shares_to_buy * price
    if total_cost + cost <= 100:
        selected_stock_details.append((stock, price, shares_to_buy, prob))
        total_cost += cost

# Display final selection
for stock, price, shares, prob in selected_stock_details:
    print(f"Stock: {stock}, Price: ${price:.2f}, Shares: {shares}, Probability of Profit: {prob*100:.2f}%")

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import yfinance as yf
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Sample stock tickers
tickers = ['AAPL', 'MSFT', 'GOOGL']

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Stock Price Viewer", style={'textAlign': 'center'}),
    
    # Dropdown for selecting a stock ticker
    dcc.Dropdown(
        id='stock-dropdown',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value='AAPL',  # Default value
        style={'width': '50%', 'margin': '0 auto'}
    ),
    
    # Graph to display the stock price
    dcc.Graph(id='stock-graph')
])

# Callback to update the graph based on the selected stock
@app.callback(
    Output('stock-graph', 'figure'),
    [Input('stock-dropdown', 'value')]
)
def update_graph(selected_ticker):
    # Fetch stock data
    stock_data = yf.Ticker(selected_ticker).history(period='6mo')
    stock_data['Date'] = stock_data.index
    
    # Create a line graph
    figure = {
        'data': [
            {'x': stock_data['Date'], 'y': stock_data['Close'], 'type': 'line', 'name': selected_ticker}
        ],
        'layout': {
            'title': f"{selected_ticker} Stock Price",
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'Close Price'}
        }
    }
    return figure

# Run the app
if __name__ == '__main__':
    # Use the `use_reloader=False` argument to prevent the app from opening multiple tabs
    app.run_server(debug=True, use_reloader=False)
